In [76]:
from protobufs.coldwallet_pb2 import CreateKeyPairsRequest, CreateKeyPairsResponse, PermuteKeyPairsRequest, PermuteKeyPairsResponse
import petlib.ec, petlib.bn

import zksk

from coldwallet.utils.nizk import deserialize_nizk

SECP256K1_CURVE_ID = 714
SECP256K1_GROUP = petlib.ec.EcGroup(nid=SECP256K1_CURVE_ID)
SECP256K1_GENERATOR = SECP256K1_GROUP.generator()

In [61]:
N = 1_000
req = CreateKeyPairsRequest(numKeys=N)
infile_name = f"create_{N}_keys_req.protobuf"
with open(infile_name, "wb") as f:
    f.write(req.SerializeToString())

In [62]:
outfile_name = f"create_{N}_keys_resp.protobuf"
!python3 -m coldwallet.coldwallet --database sqlite:///test.db --in $infile_name --out $outfile_name CreateKeyPairs

In [63]:
key_pair_resp = CreateKeyPairsResponse()
with open(outfile_name, "rb") as f:
    key_pair_resp.ParseFromString(f.read())

In [64]:
key_pair_resp.publicKeys[0].id.hex(), key_pair_resp.publicKeys[0].key.hex()

('dca329a1728d41f7975cfa523fb4e34e',
 '03a1e23b487dfe6e3cacfb7d1a6b98e998f49861981b9d6665bf8e6ad00a829d5f')

In [69]:
ids=[getattr(key_pair_resp.publicKeys[i],'id') for i in range(len(key_pair_resp.publicKeys))]

permute_req = PermuteKeyPairsRequest(
    ids=[ids[0]]
)
infile_name = f"permute_{N}_keys_req.protobuf"
with open(infile_name, "wb") as f:
    f.write(permute_req.SerializeToString())

In [70]:
outfile_name = f"permute_{N}_keys_resp.protobuf"
!python3 -m coldwallet.coldwallet --database sqlite:///test.db --in $infile_name --out $outfile_name PermuteKeyPairs

In [72]:
resp = PermuteKeyPairsResponse()
with open(outfile_name, "rb") as f:
    resp.ParseFromString(f.read())

permutation = resp.keyPermutations[0]
Y = petlib.ec.EcPt.from_binary(permutation.Y, SECP256K1_GROUP)
permutation.Y, key_pair_resp.publicKeys[0].key
assert Y == petlib.ec.EcPt.from_binary(key_pair_resp.publicKeys[0].key, SECP256K1_GROUP)

In [77]:
x_prime = petlib.bn.Bn.from_binary(permutation.XPrime)
Y_prime = x_prime * SECP256K1_GENERATOR
k_s = zksk.Secret(name="k_s")
stmt = zksk.DLRep(Y_prime, k_s * Y)
nizk = deserialize_nizk(permutation.nizk)
assert stmt.verify(nizk)